In [3]:
import time

import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm, tree
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, f1_score

In [4]:
coverages = ["A", "B", "AB"]
sections = [1, 2]
fields = ["10-G1", "13-D4"]

In [38]:
def select_df(cov, coverage, sect, section, fiel, field):
  if coverage in cov and section == sect and field == fiel:
    return True
  else:
    return False

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DOPO/Assegno di ricerca/6_ML/results.csv")

In [ ]:
def run_classifier(classifier, d, target):

  scoring = {'f1' : make_scorer(f1_score, average='macro')}
  scores = cross_validate(classifier, X, y, scoring=scoring)

  return scores["test_f1"]

In [ ]:
def run_ML(clsf, data, type_val):

  r = dict()

  if type_val:
    features = ['books', 'articles', 'other_pubbs']
  else:
    features = []

  features.extend(['cand', 'co-au', 'cand_comm', 'comm_cand', 'BC', 'CC', 'cand_other', 'other_cand', 'comm_other', 'other_comm'])
  r["no id, cop, other, comm"] = run_classifier(clsf, data[features], d['esito'])

  features.append('other')
  r["no id, cop, comm"] = run_classifier(clsf, data[features], d['esito'])

  features.pop()
  features.append('comm')
  r["no id, cop, other"] = run_classifier(clsf, data[features], d['esito'])

  features.append('other')
  r["no id, cop"]  = run_classifier(clsf, data[features], d['esito'])


  return r

In [ ]:
def results_dict(clf_type, cov_list, sect_list, field_list)

ML = dict()
if clf_type == "SVM":
  clf = svm.SVC(kernel='linear', C=1, random_state=0)
elif clf_type == "Tree"
  clf = tree.DecisionTreeClassifier()

  for cov in cov_list:
    ML[cov] = dict()
    for sect in sect_list:
      ML[cov][sect] = dict()
      for fiel in field_list:
        ML[cov][sect][fiel] = dict()
        data = df.loc[df.apply(lambda x: select_df(cov, x["coverage"], sect, x["section"], field, x["field"]), axis=1)]

        ML[cov][sect][fiel]["ANVUR"] = run_classifier(clf, data[['I1', 'I2', 'I3']], data['esito'])
        ML[cov][sect][fiel]["with type"] = run_ML(clf, data, True)
        ML[cov][sect][fiel]["no type"] = run_ML(clf, data, False)

In [ ]:
SVM_results = results_dict("SVM", coverages, sections, fields)

In [ ]:
Tree_results = results_dict("Tree", coverages, sections, fields)